In [1]:
import pandas as pd
pd.options.display.max_rows=6

import re
import sys

# match tables

In [2]:
sites = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_sites_20170713_20170714_051817_583563386.csv')
activity_cats = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_activity_cats_20170803_20170804_051931_592457785.csv')
paid_search = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_paid_search_20170730_20170731_032032_590713964.csv')

# activity

In [3]:
columns_activity = ['Event Time', 'User ID', 'Site ID (DCM)', 
                    'Activity ID', 'Total Conversions', 'Segment Value 1', 'Total Revenue']
dtype_activity = {'Event Time': int, 'User ID': str, 'Site ID (DCM)': int, 'Activity ID': int, 
                  'Total Conversions': int}

activity = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/activity_callaway_5_campaigns.csv',
                       usecols=columns_activity, dtype=dtype_activity)

activity['Total Revenue'] = activity['Total Revenue'].apply(lambda x: x*1000000)

activity = activity.merge(paid_search[['Paid Search Legacy Keyword ID', 'Paid Search Keyword']].drop_duplicates(), 
                    left_on='Segment Value 1', right_on='Paid Search Legacy Keyword ID', how='left')

In [5]:
activity = activity[activity['User ID'] != '0']

activity = activity.merge(sites[['Site ID (DCM)', 'Site (DCM)']], on='Site ID (DCM)', how='left')
activity = activity.merge(activity_cats[['Activity ID', 'Activity']], on='Activity ID', how='left')
activity = activity[activity['Activity'].str.contains('Callaway')]

hva1 = ['Callaway - About Us (Who We Are)',
        'Callaway - Create Account',
        'Callaway - Subscription (Join Our Email List)',
        'Callaway - Subscription Success']

hva2 = ['Callaway - Big & Tall',
        'Callaway - Clearance',
        'Callaway - Unsubscribe - HVA 2',
        'Callaway - Search - HVA2',
        'Callaway - Big & Tall_Belts',
        'Callaway - Big & Tall_Outerwear',
        'Callaway - Big & Tall_Pants-Shorts',
        'Callaway - Big & Tall_Polos',
        'Callaway - Clearance_Big & Tall Clearance',
        'Callaway - Clearance_Mens Clearance',
        'Callaway - Clearance_Womens Clearance',
        'Callaway - Features_Best-Sellers',
        'Callaway - Features_Callaway X',
        'Callaway - Features_New Arrivals',
        'Callaway - Features_Opti-Series',
        'Callaway - Men Golf Shoes',
        'Callaway - Men_Belts',
        'Callaway - Men_Outerwear',
        'Callaway - Men_Pants',
        'Callaway - Men_Polos',
        'Callaway - Men_Shorts',
        'Callaway - Men_Standard Collection',
        'Callaway - Products - Belts',
        'Callaway - Products - Jacket',
        'Callaway - Products - Jackets',
        'Callaway - Products - Men Golf Shoes',
        'Callaway - Products - Pants',
        'Callaway - Products - Polo',
        'Callaway - Products - Polos',
        'Callaway - Products - Short',
        'Callaway - Products - Shorts',
        'Callaway - Products - Skorts',
        'Callaway - Products - Sweaters',
        'Callaway - Products - Vests',
        'Callaway - Products - Women Golf Shoes',
        'Callaway - Women Golf Shoes',
        'Callaway - Women_Belts',
        'Callaway - Women_Outerwear',
        'Callaway - Women_Pants',
        'Callaway - Women_Polos',
        'Callaway - Women_Skorts & Shorts',
        'Callaway - Women_Standard Collection']

hva3 = ['Callaway - Add to Cart - HVA 3',
        'Callaway - Order Status (Orders)',
        'Callaway - Billing_Payment',
        'Callaway - Check Out',
        'Callaway - Order Review',
        'Callaway - Paypal',
        'Callaway - Secure Checkout',
        'Callaway - Shipping']

iterm_sold = ['Callaway - Thank You Page Items Sold']
transaction = ['Callaway - Thank You Page Transactions']

activity_to_cat = dict()
for x in hva1:
    activity_to_cat[x] = 'HVA1'
    
for x in hva2:
    activity_to_cat[x] = 'HVA2'

for x in hva3:
    activity_to_cat[x] = 'HVA3'

for x in iterm_sold:
    activity_to_cat[x] = 'iterm_sold'
    
for x in transaction:
    activity_to_cat[x] = 'transaction'

activity['activity_cat'] = activity['Activity'].apply(lambda x : activity_to_cat.get(x, 'others'))

activity['total_hva'] = activity.apply(lambda row: row['Total Conversions'] if row['activity_cat'] in {'HVA1', 'HVA2', 'HVA3'} else 0, axis=1)

In [9]:
activity['activity_cat'] = activity['activity_cat'] + activity['Paid Search Keyword'].apply(lambda x: '_'+x if type(x) == str else '')

# click

In [11]:
columns_click = ['Event Time', 'User ID', 'Site ID (DCM)', 'Segment Value 1']
dtype_click = {'Event Time': int, 'User ID': str, 'Site ID (DCM)': int}
click = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/click_callaway_5_campaigns.csv',
                    usecols=columns_click, dtype=dtype_click)

click = click[click['User ID'] != '0']

click = click.merge(sites[['Site ID (DCM)', 'Site (DCM)']], on='Site ID (DCM)', how='left')

click = click.merge(paid_search[['Paid Search Legacy Keyword ID', 'Paid Search Keyword']].drop_duplicates(), 
                    left_on='Segment Value 1', right_on='Paid Search Legacy Keyword ID', how='left')
#click['Paid Search Keyword'] = click['Paid Search Keyword'].fillna('empty')

def brand_or_non(x, brand):
    if type(x) == str:
        if brand in x:
            return 'branded_'
        else:
            return 'nonbranded_'
    else:
        return ''
click['brand_or_non'] = click['Paid Search Keyword'].apply(lambda x: brand_or_non(x, 'callaway'))

click['activity_cat'] = 'click: '+click['Site (DCM)']
click['activity_cat'] = click['activity_cat'] + click['Paid Search Keyword'].apply(lambda x: '_'+x if type(x) == str else '')
click

,Event Time,User ID,Site ID (DCM),Segment Value 1,Site (DCM),Paid Search Legacy Keyword ID,Paid Search Keyword,brand_or_non,activity_cat
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,1695745,NaN,QuantCast,NaN,NaN,,click: QuantCast
1,1490969647208748,AMsySZagAWzbVDPRBTEAXJN7YQ7s,1695745,NaN,QuantCast,NaN,NaN,,click: QuantCast
2,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,1695745,NaN,QuantCast,NaN,NaN,,click: QuantCast
...,...,...,...,...,...,...,...,...,...
131385,1501593776016244,AMsySZYBOtUGZwisXSFPnBoaEica,1234535,1.98789e+10,DART Search : Google,NaN,NaN,,click: DART Search : Google
131386,1501594253008116,AMsySZYB-vjNBON3A76FFeojxLRd,3446099,NaN,Criteo,NaN,NaN,,click: Criteo
131387,1501596360142324,AMsySZZ0nfhuYLaGNOGkVCE-7Go3,1234535,1.98789e+10,DART Search : Google,NaN,NaN,,click: DART Search : Google


In [12]:
click.sort_values(by='Event Time', inplace=True)
users_from_non_to_brand = list()
for user_id, group in click.groupby('User ID'):
    #group.sort_values(by='Event Time', inplace=True)
    brand_or_non=list(group['brand_or_non'])
    if 'branded_' in brand_or_non and 'nonbranded_' in brand_or_non:
        brand_index = brand_or_non.index('branded_')
        non_brand_index = brand_or_non.index('nonbranded_')
        if brand_index > non_brand_index:
            kws = list(group['Paid Search Keyword'])
            sts = list(group['Site (DCM)'])
            d = {'user_id': user_id,
                 'nonbranded_kw': sts[non_brand_index]+'>>>'+kws[non_brand_index],
                 'branded_kw': sts[brand_index]+'>>>'+kws[brand_index]}
            users_from_non_to_brand.append(d)
            
users_from_nonbranded_to_branded = pd.DataFrame(users_from_non_to_brand)
users_from_nonbranded_to_branded

,branded_kw,nonbranded_kw,user_id
0,DART Search : Google>>>+callaway +ladies,DART Search : Google>>>mens golf shoes,AMsySZY1BO3tjTnQEgiD-oZixmpr
1,DART Search : Google>>>callaway golf shirts,DART Search : MSN>>>+men +polo +shirt,AMsySZY2bIRs2YglXxuf04j2NyMe
2,DART Search : Google>>>+callaway +shirt,DART Search : Google>>>+golf +shirts,AMsySZY444-nFZ1uL27C3Lr-SPJg
...,...,...,...
153,DART Search : MSN>>>callaway apparel,DART Search : Google>>>+golf +vest,AMsySZbrVP0-5U3EKYENnxWi4vuv
154,DART Search : Google>>>callaway golf women,DART Search : Google>>>+mens +golf +pants,AMsySZbtwkNV2MDS8Yr-pdNfHxWE
155,DART Search : Google>>>+callaway +golf +shoes,DART Search : Google>>>mens golf shoes,AMsySZbu0h2Nm4F6ftt1pqQOCyK4


In [13]:
target_users = set(users_from_nonbranded_to_branded['user_id'])

In [ ]:
activity[activity['User ID'].isin(target_users)]

In [14]:
merged_columns = ['Event Time', 'User ID', 'activity_cat']
merged = click[merged_columns].append(activity[merged_columns], ignore_index=True)
merged = merged[merged['User ID'].isin(target_users)]
merged.sort_values(by='Event Time', inplace=True)

merged.reset_index(inplace=True, drop=True)

merged

,Event Time,User ID,activity_cat
0,1491143667108333,AMsySZZrKd55sCCtg9YV78DSzQYw,click: DART Search : MSN_women's golf apparel
1,1491143669658472,AMsySZZrKd55sCCtg9YV78DSzQYw,others_women's golf apparel
2,1491228516342372,AMsySZZrKd55sCCtg9YV78DSzQYw,click: DART Search : MSN
...,...,...,...
10829,1501539520878857,AMsySZbhJ6fFVBIBEfVGfMMI4VmE,others_+golf +apparel
10830,1501539520883829,AMsySZbhJ6fFVBIBEfVGfMMI4VmE,others_+golf +apparel
10831,1501591738358622,AMsySZbhJ6fFVBIBEfVGfMMI4VmE,click: DART Search : MSN_golf apparel


In [15]:
def path_list_to_str(path):
    '''
    Args: 
        path: list, ['a', 'a', 'b', 'c', 'c', 'c']
    Returns:
        str, 'a(2) -> b(1) -> c(3)'
    '''
    path = [x for x in path if x != 'others']
    #if 'transaction' in path:
    #    path = path[: path.index('transaction')+1]
    curr = path[0]
    count = 0
    result = list()
    for x in path:
        if x == 'others':
            continue
        elif x == curr:
            count += 1
        else:
            result.append('%s(%d)'%(curr, count))
            curr = x
            count = 1
    result.append('%s(%d)'%(curr, count))
    return ' -> '.join(result)

path_list_to_str(['a', 'a', 'others', 'b', 'others', 'c', 'c',  'c', 'others'])

'a(2) -> b(1) -> c(3)'

In [17]:
merged_groups_by_id = merged.groupby('User ID')

user_path_dict = dict()
for user, group in merged_groups_by_id:
    try:
        path_list = list(group['activity_cat'])
        path = path_list_to_str(path_list)
        #print('%s >>>> %s'%(user, path))
        user_path_dict[user] = path
    except Exception as e:
        print(user, e)

user_path = pd.DataFrame({'user_id': list(user_path_dict.keys()), 'path': list(user_path_dict.values())})

user_path

,path,user_id
0,click: DART Search : Google_mens golf shoes(1)...,AMsySZY1BO3tjTnQEgiD-oZixmpr
1,click: DART Search : MSN_+men +polo +shirt(1) ...,AMsySZY2bIRs2YglXxuf04j2NyMe
2,click: DART Search : Google_+golf +shirts(1) -...,AMsySZY444-nFZ1uL27C3Lr-SPJg
...,...,...
153,click: DART Search : Google_+golf +vest(1) -> ...,AMsySZbrVP0-5U3EKYENnxWi4vuv
154,click: DART Search : Google_+mens +golf +pants...,AMsySZbtwkNV2MDS8Yr-pdNfHxWE
155,click: DART Search : Google_mens golf shoes(1)...,AMsySZbu0h2Nm4F6ftt1pqQOCyK4


In [24]:
transaction_by_user = list()
for u in target_users:
    df = activity[activity['User ID'] == u]
    transaction = df[df['Activity']=='Callaway - Thank You Page Transactions']['Total Conversions'].sum()
    item_sold = df[df['Activity']=='Callaway - Thank You Page Items Sold']['Total Conversions'].sum()
    revenue = df['Total Revenue'].sum()
    conversion = df['Total Conversions'].sum() - transaction
    transaction_by_user.append({'user_id' : u,
                                'conversion': conversion,
                                'transaction': transaction,
                                'item_sold': item_sold,
                                'revenue': revenue
                               })

In [25]:
pd.DataFrame(transaction_by_user)

,conversion,item_sold,revenue,transaction,user_id
0,9,0,0.0,0,AMsySZZxub--NqbDp4CxCHfnXyEV
1,47,0,0.0,0,AMsySZZ_Y11iKoSE5e0PqhyFD_cI
2,91,2,620.0,1,AMsySZbl-B3Ip3W2DadWOXBLXnt8
...,...,...,...,...,...
153,119,0,0.0,0,AMsySZahoWczqHfHZ2XlccO35pLX
154,44,0,0.0,0,AMsySZbB8-8_UgV4xaDliwBOyTM9
155,63,0,0.0,0,AMsySZb52T_r2IhpvcFO8IB3D0hp


In [26]:
result = users_from_nonbranded_to_branded.merge(user_path, on='user_id', how='left')
result = result.merge(pd.DataFrame(transaction_by_user), on='user_id', how='left')

In [27]:
result.to_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/non_branded_to_branded.csv', index=False)